In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import json
import os

class MultiCameraTennisCalibration:
    """
    Multi-camera calibration using tennis court keypoints.
    Demonstrates professional computer vision techniques for sports analysis with multiple cameras.
    """
    
    def __init__(self):
        # Standard tennis court dimensions (meters)
        self.court_length = 12.0  # Simplified court length
        self.court_width = 6.0    # Simplified court width
        self.service_line_distance = 3.0  # Distance from net to service line
        
        # Define 3D court keypoints in world coordinates (court-centered)
        self.world_keypoints_3d = self.create_court_keypoints_3d()
        
    def create_court_keypoints_3d(self):
        """Create standard tennis court keypoints in 3D world coordinates"""
        keypoints = {}
        
        # Court corners (ground level z=0)
        keypoints['back_left'] = [-self.court_length/2, -self.court_width/2, 0]
        keypoints['back_right'] = [-self.court_length/2, self.court_width/2, 0]
        keypoints['front_left'] = [self.court_length/2, -self.court_width/2, 0]
        keypoints['front_right'] = [self.court_length/2, self.court_width/2, 0]
        
        # Net line intersections
        keypoints['net_left'] = [0, -self.court_width/2, 0]
        keypoints['net_right'] = [0, self.court_width/2, 0]
        keypoints['net_center'] = [0, 0, 0]
        
        # Service line intersections
        keypoints['service_back_left'] = [-self.service_line_distance, -self.court_width/2, 0]
        keypoints['service_back_right'] = [-self.service_line_distance, self.court_width/2, 0]
        keypoints['service_front_left'] = [self.service_line_distance, -self.court_width/2, 0]
        keypoints['service_front_right'] = [self.service_line_distance, self.court_width/2, 0]
        
        # Center service line intersections
        keypoints['center_service_back'] = [-self.service_line_distance, 0, 0]
        keypoints['center_service_front'] = [self.service_line_distance, 0, 0]
        
        return keypoints

    def create_full_court_geometry_3d(self):
        """Create court geometry for overlay visualization"""
        court_lines = []
        
        # Court boundary rectangle
        court_lines.extend([
            [[-self.court_length/2, -self.court_width/2, 0], [-self.court_length/2, self.court_width/2, 0]],  # Left baseline
            [[self.court_length/2, -self.court_width/2, 0], [self.court_length/2, self.court_width/2, 0]],    # Right baseline  
            [[-self.court_length/2, -self.court_width/2, 0], [self.court_length/2, -self.court_width/2, 0]],  # Bottom sideline
            [[-self.court_length/2, self.court_width/2, 0], [self.court_length/2, self.court_width/2, 0]]     # Top sideline
        ])
        
        # Net line
        court_lines.append([[0, -self.court_width/2, 0], [0, self.court_width/2, 0]])
        
        # Service lines
        court_lines.extend([
            [[-self.service_line_distance, -self.court_width/2, 0], [-self.service_line_distance, self.court_width/2, 0]],  # Left service
            [[self.service_line_distance, -self.court_width/2, 0], [self.service_line_distance, self.court_width/2, 0]]     # Right service
        ])
        
        # Center service line
        court_lines.append([[-self.service_line_distance, 0, 0], [self.service_line_distance, 0, 0]])
        
        return court_lines
    
    def load_detections_from_simulation(self, simulation_data_path="../utils/data/simulated/synthetic_scene.npz"):
        """
        Load detections from your tennis simulation data
        """
        # Load simulation data
        data = np.load(simulation_data_path, allow_pickle=True)
        camera_poses = data['camera_poses']
        intrinsics = data['intrinsics']
        
        print(f"Loaded simulation data with {len(camera_poses)} cameras")
        
        # For each camera, create synthetic keypoint detections
        multi_camera_detections = {}
        
        for cam_idx in range(len(camera_poses)):
            R = camera_poses[cam_idx][:3, :3]
            t = camera_poses[cam_idx][:3, 3]
            K = intrinsics
            
            # Project all court keypoints to this camera
            detections = self.create_synthetic_detections_for_camera(K, R, t, noise_std=1.0)
            
            if len(detections) >= 6:  # Need at least 6 points for PnP
                multi_camera_detections[f'camera_{cam_idx+1}'] = {
                    'detections': detections,
                    'K_true': K.copy(),
                    'R_true': R.copy(),
                    't_true': t.copy()
                }
                print(f"Camera {cam_idx+1}: {len(detections)} keypoints visible")
            else:
                print(f"Camera {cam_idx+1}: Only {len(detections)} keypoints visible (need ≥6)")
        
        return multi_camera_detections
            
    
    def project_points(self, K, R, t, points_3d):
        """Project 3D points to 2D using camera parameters"""
        # Transform to camera coordinates
        points_cam = (R @ points_3d.T).T + t.reshape(1, 3)
        
        points_2d = np.zeros((len(points_3d), 2))
        depths = np.zeros(len(points_3d))
        
        for i, (x, y, z) in enumerate(points_cam):
            depths[i] = z
            if z > 0.1:  # Point in front of camera
                u = K[0,0] * x/z + K[0,2]
                v = K[1,1] * y/z + K[1,2]
                points_2d[i] = [u, v]
            else:
                points_2d[i] = [np.nan, np.nan]
        
        return points_2d, depths
    
    def create_synthetic_detections_for_camera(self, K_true, R_true, t_true, noise_std=0.5):
        """Create synthetic 2D keypoint detections for a single camera"""
        # Convert world keypoints to numpy array
        keypoint_names = list(self.world_keypoints_3d.keys())
        points_3d = np.array([self.world_keypoints_3d[name] for name in keypoint_names])
        
        # Project to 2D
        points_2d_clean, depths = self.project_points(K_true, R_true, t_true, points_3d)
        
        # Filter out points behind camera or outside reasonable bounds
        valid_mask = (~np.isnan(points_2d_clean).any(axis=1)) & (depths > 0.1)
        
        # Add realistic noise only to valid points
        detections = {}
        for i, name in enumerate(keypoint_names):
            if valid_mask[i]:
                # Check if point is within image bounds (with some margin)
                x, y = points_2d_clean[i]
                if -100 <= x <= 740 and -100 <= y <= 580:  # 640x480 image with margin
                    noise = np.random.normal(0, noise_std, 2)
                    points_2d_noisy = points_2d_clean[i] + noise
                    
                    detections[name] = {
                        'point_2d': points_2d_noisy,
                        'confidence': 0.7 + 0.3 * np.random.random(),
                        'world_3d': self.world_keypoints_3d[name],
                        'clean_2d': points_2d_clean[i]
                    }
                
        return detections
    
    def create_synthetic_multi_camera_data(self):
        """Create synthetic multi-camera data if simulation data not available"""
        print("Creating synthetic multi-camera scenario...")
        
        camera_configs = [
            {'name': 'camera_1', 'pos': [0.0, -5.0, 3.5], 'target': [0.0, 0.0, 0.5]},
            {'name': 'camera_2', 'pos': [-4.0, -3.5, 3.5], 'target': [1.0, 1.5, 0.0]},
            {'name': 'camera_3', 'pos': [4.0, -3.5, 3.5], 'target': [-1.0, 1.5, 0.0]}
        ]
        
        multi_camera_detections = {}
        
        for config in camera_configs:
            # Create camera parameters
            K_true = np.array([[400, 0, 320], [0, 400, 240], [0, 0, 1]], dtype=np.float64)
            
            camera_pos = np.array(config['pos'])
            target_pos = np.array(config['target'])
            
            # Calculate camera rotation and translation
            look_vector = target_pos - camera_pos
            look_vector = look_vector / np.linalg.norm(look_vector)
            
            world_up = np.array([0, 0, 1])
            right = np.cross(look_vector, world_up)
            right = right / np.linalg.norm(right)
            up = -np.cross(right, look_vector)
            
            R_true = np.column_stack([right, up, look_vector]).T
            t_true = -R_true @ camera_pos
            
            # Generate detections for this camera
            detections = self.create_synthetic_detections_for_camera(K_true, R_true, t_true, noise_std=1.0)
            
            multi_camera_detections[config['name']] = {
                'detections': detections,
                'K_true': K_true,
                'R_true': R_true,
                't_true': t_true
            }
            
            print(f"{config['name']}: {len(detections)} keypoints detected")
        
        return multi_camera_detections
    
    def calibrate_single_camera_pnp(self, detections, initial_K=None):
        """Calibrate a single camera using PnP algorithm"""
        if len(detections) < 6:
            raise ValueError(f"Need at least 6 keypoints for PnP, got {len(detections)}")
            
        # Extract 2D and 3D points
        keypoint_names = list(detections.keys())
        points_2d = np.array([detections[name]['point_2d'] for name in keypoint_names], dtype=np.float32)
        points_3d = np.array([detections[name]['world_3d'] for name in keypoint_names], dtype=np.float32)
        
        # Use initial intrinsics guess if provided
        if initial_K is None:
            fx = fy = 400  # Focal length
            cx, cy = 320, 240  # Image center
            initial_K = np.array([[fx, 0, cx],
                                 [0, fy, cy],
                                 [0, 0, 1]], dtype=np.float32)
        
        # Distortion coefficients (assume minimal distortion)
        dist_coeffs = np.zeros(4, dtype=np.float32)
        
        # Solve PnP
        success, rvec, tvec = cv2.solvePnP(points_3d, points_2d, initial_K, dist_coeffs, 
                                          flags=cv2.SOLVEPNP_ITERATIVE)
        
        if not success:
            raise ValueError("PnP solution failed")
            
        # Convert rotation vector to matrix
        R_estimated, _ = cv2.Rodrigues(rvec)
        t_estimated = tvec.flatten()
        
        return initial_K, R_estimated, t_estimated, success
    
    def calibrate_multi_camera_bundle_adjustment(self, multi_camera_detections):
        """Multi-camera bundle adjustment calibration"""
        camera_names = list(multi_camera_detections.keys())
        num_cameras = len(camera_names)
        
        print(f"\nPerforming bundle adjustment for {num_cameras} cameras...")
        
        # Get initial estimates for all cameras using PnP
        initial_calibrations = {}
        for cam_name in camera_names:
            detections = multi_camera_detections[cam_name]['detections']
            try:
                K_init, R_init, t_init, success = self.calibrate_single_camera_pnp(detections)
                initial_calibrations[cam_name] = {
                    'K': K_init, 'R': R_init, 't': t_init, 'success': success
                }
                print(f"{cam_name}: PnP successful, {len(detections)} keypoints")
            except Exception as e:
                print(f"{cam_name}: PnP failed - {e}")
                return None, None
        
        def bundle_adjustment_objective(params):
            """Bundle adjustment cost function"""
            total_cost = 0
            param_idx = 0
            
            for i, cam_name in enumerate(camera_names):
                # Extract camera parameters from optimization vector
                fx = params[param_idx]
                fy = params[param_idx + 1] 
                cx = params[param_idx + 2]
                cy = params[param_idx + 3]
                rvec = params[param_idx + 4:param_idx + 7]
                tvec = params[param_idx + 7:param_idx + 10]
                param_idx += 10
                
                # Reconstruct camera matrix and pose
                K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
                R, _ = cv2.Rodrigues(rvec)
                
                # Calculate reprojection error for this camera
                detections = multi_camera_detections[cam_name]['detections']
                for name, detection in detections.items():
                    point_3d = np.array([detection['world_3d']])
                    point_2d_proj, _ = self.project_points(K, R, tvec, point_3d)
                    
                    if not np.isnan(point_2d_proj[0]).any():
                        point_2d_obs = detection['point_2d']
                        weight = detection['confidence']
                        error = weight * np.linalg.norm(point_2d_proj[0] - point_2d_obs)**2
                        total_cost += error
            
            return total_cost
        
        # Create initial parameter vector
        initial_params = []
        for cam_name in camera_names:
            calib = initial_calibrations[cam_name]
            K, R, t = calib['K'], calib['R'], calib['t']
            rvec, _ = cv2.Rodrigues(R)
            
            # Add parameters: fx, fy, cx, cy, rvec(3), tvec(3)
            cam_params = [K[0,0], K[1,1], K[0,2], K[1,2]] + rvec.flatten().tolist() + t.tolist()
            initial_params.extend(cam_params)
        
        initial_params = np.array(initial_params)
        initial_cost = bundle_adjustment_objective(initial_params)
        
        # Optimize all cameras simultaneously
        print("Running bundle adjustment optimization...")
        result = minimize(bundle_adjustment_objective, initial_params, method='L-BFGS-B',
                         options={'maxiter': 1000, 'ftol': 1e-9})
        
        # Extract optimized parameters
        optimized_calibrations = {}
        param_idx = 0
        
        for cam_name in camera_names:
            params = result.x[param_idx:param_idx+10]
            fx, fy, cx, cy = params[:4]
            rvec = params[4:7]
            tvec = params[7:10]
            
            K_opt = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
            R_opt, _ = cv2.Rodrigues(rvec)
            
            optimized_calibrations[cam_name] = {
                'K': K_opt,
                'R': R_opt,
                't': tvec,
                'initial_cost': initial_cost,
                'final_cost': result.fun
            }
            param_idx += 10
        
        return optimized_calibrations, result
    
    def evaluate_multi_camera_calibration(self, multi_camera_detections, calibrations):
        """Evaluate calibration quality across all cameras"""
        evaluation = {}
        
        for cam_name, cam_data in multi_camera_detections.items():
            if cam_name not in calibrations:
                continue
                
            detections = cam_data['detections']
            K = calibrations[cam_name]['K']
            R = calibrations[cam_name]['R']
            t = calibrations[cam_name]['t']
            
            # Calculate errors
            errors = []
            for name, detection in detections.items():
                point_3d = np.array([detection['world_3d']])
                point_2d_proj, _ = self.project_points(K, R, t, point_3d)
                
                if not np.isnan(point_2d_proj[0]).any():
                    point_2d_obs = detection['point_2d']
                    error = np.linalg.norm(point_2d_proj[0] - point_2d_obs)
                    errors.append(error)
            
            if errors:
                evaluation[cam_name] = {
                    'mean_error': np.mean(errors),
                    'max_error': np.max(errors),
                    'std_error': np.std(errors),
                    'num_keypoints': len(errors)
                }
            
            # Compare with ground truth if available
            if 'K_true' in cam_data:
                K_true = cam_data['K_true']
                R_true = cam_data['R_true']
                t_true = cam_data['t_true']
                
                # Intrinsic parameter errors
                fx_error = abs(K[0,0] - K_true[0,0])
                fy_error = abs(K[1,1] - K_true[1,1])
                cx_error = abs(K[0,2] - K_true[0,2])
                cy_error = abs(K[1,2] - K_true[1,2])
                
                # Camera position error
                pos_true = -R_true.T @ t_true
                pos_est = -R.T @ t
                pos_error = np.linalg.norm(pos_true - pos_est)
                
                evaluation[cam_name]['ground_truth_comparison'] = {
                    'fx_error': fx_error,
                    'fy_error': fy_error,
                    'cx_error': cx_error,
                    'cy_error': cy_error,
                    'position_error_m': pos_error
                }
        
        return evaluation
    
    def create_court_overlay(self, multi_camera_detections, calibrations, 
                                         court_images_path="/sports-vision/utils/data/simulated/camera_views"):
        """
        Create overlay visualization comparing original court images with reprojected court lines
        """
        overlay_visualizations = {}
        
        # Get complete court geometry for overlay
        court_lines_3d = self.create_full_court_geometry_3d()
        
        for cam_name, cam_data in multi_camera_detections.items():
            if cam_name not in calibrations:
                continue
                
            # Try to load original court image
            cam_number = cam_name.split('_')[1]
            original_image_path = os.path.join(court_images_path, f"cam_{cam_number}_tennis_view.png")
            
            if os.path.exists(original_image_path):
                # Load original synthetic image
                original_image = cv2.imread(original_image_path)
                print(f"Loaded original image: {original_image_path}")
            else:
                # Create blank court-like image
                original_image = self.create_court_background_image()
                print(f"Created synthetic court background for {cam_name}")
            
            # Create overlay image
            overlay_image = original_image.copy()
            
            # Get calibrated camera parameters
            K = calibrations[cam_name]['K']
            R = calibrations[cam_name]['R']
            t = calibrations[cam_name]['t']
            
            # Project and draw complete court lines using calibrated parameters
            print(f"Projecting court lines for {cam_name}...")
            for line_3d in court_lines_3d:
                line_points_3d = np.array(line_3d)
                line_points_2d, depths = self.project_points(K, R, t, line_points_3d)
                
                # Draw line if both endpoints are in front of camera
                if np.all(depths > 0.1) and not np.isnan(line_points_2d).any():
                    pt1 = tuple(line_points_2d[0].astype(int))
                    pt2 = tuple(line_points_2d[1].astype(int))
                    
                    # Draw reprojected court line in bright color (magenta)
                    cv2.line(overlay_image, pt1, pt2, (255, 0, 255), 3)
            
            # Draw keypoint detections and reprojections
            detections = cam_data['detections']
            
            # Draw detected keypoints (red circles)
            for name, detection in detections.items():
                pt = tuple(detection['point_2d'].astype(int))
                if 0 <= pt[0] < overlay_image.shape[1] and 0 <= pt[1] < overlay_image.shape[0]:
                    cv2.circle(overlay_image, pt, 6, (0, 0, 255), -1)  # Red
                    cv2.putText(overlay_image, name[:4], (pt[0]+10, pt[1]), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            
            # Draw reprojected keypoints (green circles)
            for name, detection in detections.items():
                point_3d = np.array([detection['world_3d']])
                point_2d_proj, _ = self.project_points(K, R, t, point_3d)
                
                if not np.isnan(point_2d_proj[0]).any():
                    pt_proj = tuple(point_2d_proj[0].astype(int))
                    if 0 <= pt_proj[0] < overlay_image.shape[1] and 0 <= pt_proj[1] < overlay_image.shape[0]:
                        cv2.circle(overlay_image, pt_proj, 4, (0, 255, 0), -1)  # Green
                        
                        # Draw error vector (cyan line)
                        pt_det = tuple(detection['point_2d'].astype(int))
                        cv2.line(overlay_image, pt_det, pt_proj, (255, 255, 0), 2)
            
            # Add calibration quality info
            if cam_name in calibrations:
                eval_data = self.evaluate_single_camera(cam_data, calibrations[cam_name])
                
                # Add text overlay with calibration info
                info_text = [
                    f"{cam_name.upper()} - Calibration Overlay",
                    f"Reprojection Error: {eval_data['mean_error']:.2f}px",
                    f"Keypoints: {eval_data['num_keypoints']}",
                    "Magenta: Reprojected Court",
                    "Red: Detected Keypoints", 
                    "Green: Reprojected Keypoints",
                    "Cyan: Reprojection Errors"
                ]
                
                for i, text in enumerate(info_text):
                    y_pos = 30 + i * 25
                    cv2.putText(overlay_image, text, (10, y_pos), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            overlay_visualizations[cam_name] = overlay_image
            
        return overlay_visualizations
    
    def evaluate_single_camera(self, cam_data, calibration):
        """Evaluate single camera calibration"""
        detections = cam_data['detections']
        K = calibration['K']
        R = calibration['R']
        t = calibration['t']
        
        errors = []
        for name, detection in detections.items():
            point_3d = np.array([detection['world_3d']])
            point_2d_proj, _ = self.project_points(K, R, t, point_3d)
            
            if not np.isnan(point_2d_proj[0]).any():
                point_2d_obs = detection['point_2d']
                error = np.linalg.norm(point_2d_proj[0] - point_2d_obs)
                errors.append(error)
        
        return {
            'mean_error': np.mean(errors) if errors else 0,
            'max_error': np.max(errors) if errors else 0,
            'num_keypoints': len(errors)
        }
    
    def create_court_background_image(self, image_size=(640, 480)):
        """Create a synthetic court background if original image not available"""
        image = np.zeros((image_size[1], image_size[0], 3), dtype=np.uint8)
        
        # Sky gradient
        for y in range(int(image_size[1] * 0.4)):
            intensity = 180 + int((y / (image_size[1] * 0.4)) * 55)
            blue = min(255, intensity + 75)
            green = min(255, intensity + 26) 
            red = min(255, intensity - 45)
            image[y, :] = [blue, green, red]
        
        # Court surface (simplified)
        court_color = (34, 139, 34)  # Green
        cv2.rectangle(image, (0, int(image_size[1]*0.4)), (image_size[0], image_size[1]), court_color, -1)
        
        return image

def multi_camera_calibration():
    """Complete demonstration of multi-camera calibration with court overlay visualization"""
    
    # Initialize calibration system
    calibrator = MultiCameraTennisCalibration()
    
    # 1. Load multi-camera detections
    print("\n1. Loading multi-camera court keypoint detections...")
    multi_camera_detections = calibrator.load_detections_from_simulation()
    
    if not multi_camera_detections:
        print("No camera detections available!")
        return
    
    # 2. Perform multi-camera bundle adjustment
    print("\n2. Performing multi-camera bundle adjustment calibration...")
    
    calibrations, optimization_result = calibrator.calibrate_multi_camera_bundle_adjustment(multi_camera_detections)
    
    if calibrations is None:
        print("Multi-camera calibration failed!")
        return
        
    print(f"Bundle adjustment converged: {optimization_result.success}")
    print(f"Initial cost: {list(calibrations.values())[0]['initial_cost']:.2f}")
    print(f"Final cost: {optimization_result.fun:.2f}")
        
    # 3. Evaluate calibration results
    print("\n3. Evaluating calibration results...")
    evaluation = calibrator.evaluate_multi_camera_calibration(multi_camera_detections, calibrations)
    
    # 4. Create court overlay visualizations
    print("\n4.Creating court overlay visualizations...")
    overlay_visualizations = calibrator.create_court_overlay(
        multi_camera_detections, calibrations)
    
    for cam_name, overlay_image in overlay_visualizations.items():
        filename = f"tennis_calibration_overlay_{cam_name}.png"
        cv2.imwrite(filename, overlay_image)
    
    # 5. Print results summary
    print("\n5.Calibration Results Summary:")
    for cam_name, eval_data in evaluation.items():
        print(f"\n{cam_name.upper()}:")
        print(f"Mean reprojection error: {eval_data['mean_error']:.3f} pixels")
        print(f"Max reprojection error: {eval_data['max_error']:.3f} pixels")
        print(f"Keypoints used: {eval_data['num_keypoints']}")
        
        if 'ground_truth_comparison' in eval_data:
            gt = eval_data['ground_truth_comparison']
            print(f"Ground truth errors:")
            print(f"Focal length: fx={gt['fx_error']:.2f}, fy={gt['fy_error']:.2f}")
            print(f"Principal point: cx={gt['cx_error']:.2f}, cy={gt['cy_error']:.2f}")
            print(f"Position: {gt['position_error_m']:.3f}m")
    
    # 6. Save comprehensive results
    results = {
        'overlay_visualization_info': {
            'description': 'Court overlay showing original synthetic images with reprojected court lines',
            'color_coding': {
                'magenta_lines': 'Reprojected court lines using calibrated parameters',
                'red_circles': 'Detected court keypoints (with noise)',
                'green_circles': 'Reprojected keypoints using calibrated parameters',
                'cyan_lines': 'Reprojection error vectors (detected to reprojected)',
                'original_court': 'Original synthetic tennis court from simulation'
            },
            'interpretation': {
                'good_calibration': 'Magenta lines should align well with original court lines',
                'keypoint_accuracy': 'Short cyan error lines indicate good calibration',
                'overall_quality': 'Visual alignment shows calibration accuracy'
            }
        },
        'multi_camera_calibration': {},
        'evaluation': evaluation,
        'optimization_info': {
            'success': optimization_result.success,
            'final_cost': float(optimization_result.fun),
            'message': optimization_result.message
        },
        'techniques_demonstrated': [
            'Multi-camera bundle adjustment',
            'Court overlay visualization',
            'Reprojection error analysis',
            'Ground truth validation',
            'Visual calibration assessment'
        ]
    }
    
    for cam_name, calib in calibrations.items():
        camera_position = -calib['R'].T @ calib['t']
        results['multi_camera_calibration'][cam_name] = {
            'intrinsics': {
                'fx': float(calib['K'][0,0]),
                'fy': float(calib['K'][1,1]),
                'cx': float(calib['K'][0,2]),
                'cy': float(calib['K'][1,2])
            },
            'extrinsics': {
                'rotation_matrix': calib['R'].tolist(),
                'translation_vector': calib['t'].tolist(),
                'camera_position': camera_position.tolist()
            }
        }
    
    with open('tennis_calibration_overlay_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print("\n Saved overlay results: tennis_calibration_overlay_results.json")
    
    print("\n Multi-camera calibration with court overlay completed successfully!")
    print("Check the overlay images to visually assess calibration quality!")
    
    return calibrator, calibrations, multi_camera_detections, overlay_visualizations


In [ ]:
multi_camera_calibration()